In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/Shareddrives/IF4074_Tubes 2_13519074')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# CNN

## Imports

In [2]:
import os
from classes.layers.Conv2D import Conv2D
from classes.layers.Dense import Dense
from classes.layers.Flatten import Flatten
from classes.layers.Input import InputLayer
from classes.models.Sequential import Sequential
from classes.utils.ImageConvert import ImageConvert
from classes.losses.BinaryCrossentropy import BinaryCrossentropy

import numpy as np

## Data Preprocessing

In [26]:
ic_train = ImageConvert(
        rotate=30.,
        rescale=1./255.)
data_train_gen = ic_train.from_directory(os.path.join('/content/gdrive/Shareddrives/IF4074_Tubes 2_13519074/data','Cat-Dog','train'), (256,256), mode='binary', color_mode='rgb')

ic_test = ImageConvert(
        rotate=30.,
        rescale=1./255.)
data_test_gen = ic_test.from_directory(os.path.join('/content/gdrive/Shareddrives/IF4074_Tubes 2_13519074/data','Cat-Dog','test'), (256,256), mode='binary', color_mode='rgb')

In [4]:
X_train = []
y_train = []

for i in range(len(data_train_gen)):
    step = next(data_train_gen)
    X_train.append(step['data'])
    y_train.append(step['label'])

In [14]:
X_test = []
y_test = []

for i in range(len(data_test_gen)):
    step = next(data_test_gen)
    X_test.append(step['data'])
    y_test.append(step['label'])

## MODEL

### Instantiating

In [5]:
def generate_n_model(n):
    models = []
    for _ in range(n):
        model = Sequential([
                InputLayer(input_shape=(256,256,3)),
                Conv2D(2, (16, 16), activation='relu'),
                Flatten(),
                Dense(16, activation='relu'),
                Dense(1, activation='sigmoid')
            ],  name=f'Model_{_}')

        model.compile(loss=BinaryCrossentropy())
        models.append(model)
    return models

In [6]:
models = generate_n_model(10)

In [25]:
models[0].summary()

Name          Type        Output Shape           Num of Params
------------  ----------  -------------------  ---------------
InputLayer    InputLayer  (None, 256, 256, 3)                0
Conv2D        Conv2D      (None, 242, 242, 2)             1538
FlattenLayer  Flatten     (None, 117128)                     0
Dense         Dense       (None, 16)                   1874064
Dense         Dense       (None, 1)                         17
Total params:  1875619
Trainable params:  1875619
Non-trainable params:  0


### Training

In [7]:
def cross_validation_split(data, label, n):
    data_split = [list(_) for _ in np.array_split(data, n)]
    label_split = [list(_) for _ in np.array_split(label, n)]
    
    train_split = []
    valid_split = []
    label_train_split = []
    label_valid_split = []

    for i in range(len(data_split)):
        # Separate validation k fold split
        for data in data_split[i]:
            valid_split.append(data)
        for label in label_split[i]:
            label_valid_split.append(label)
        # Union train k fold split
        temp_train_data = []
        for data in data_split[:i]:
            # for data in set:
            temp_train_data += list(data)
        for data in data_split[i+1:]:
            # for data in set:
            temp_train_data += list(data)
        train_split.append(temp_train_data)
        
        temp_train_label = []
        for set in label_split[:i]:
            for label in set:
                temp_train_label.append(label)
        for set in label_split[i+1:]:
            for label in set:
                temp_train_label.append(label)
        label_train_split.append(temp_train_label)
    
    return train_split, valid_split, label_train_split, label_valid_split

train_split, valid_split,\
    label_train_split, label_valid_split = cross_validation_split(X_train, y_train, 10)

In [8]:
for i in range(len(models)):
    print(models[i].name)
    models[i].fit(train_split[i], label_train_split[i], 2, 1, 0.2)
    print()
    print()

Model_0
Epochs 1/1
Step: 81/81


Model_1
Epochs 1/1
Step: 81/81
Step: 82/81


Model_2
Epochs 1/1
Step: 81/81
Step: 82/81


Model_3
Epochs 1/1
Step: 81/81
Step: 82/81


Model_4
Epochs 1/1
Step: 81/81
Step: 82/81


Model_5
Epochs 1/1
Step: 81/81
Step: 82/81


Model_6
Epochs 1/1
Step: 81/81
Step: 82/81


Model_7
Epochs 1/1
Step: 81/81
Step: 82/81


Model_8
Epochs 1/1
Step: 81/81
Step: 82/81


Model_9
Epochs 1/1
Step: 81/81
Step: 82/81




### Testing

In [9]:
# TEST
for model in models:
    print(model.name)
    model.predict(data_test_gen)

Model_0
Step: 40/40 | loss: -7.7124742353307765
Model_1
Step: 40/40 | loss: -7.7124742353307765
Model_2
Step: 40/40 | loss: -7.7124742353307765
Model_3
Step: 40/40 | loss: -7.7124742353307765
Model_4
Step: 40/40 | loss: -7.7124742353307765
Model_5
Step: 40/40 | loss: -7.7124742353307765
Model_6
Step: 40/40 | loss: -7.7124742353307765
Model_7
Step: 40/40 | loss: -7.7124742353307765
Model_8
Step: 40/40 | loss: -7.7124742353307765
Model_9
Step: 40/40 | loss: -7.7124742353307765


### Pick Best Model

In [10]:
model = models[2]

## SAVE & LOAD

In [11]:
model.save("cnn_model")

In [12]:
model.load("cnn_model.json")

In [15]:
model.predict(X_test)

[array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.])]

# RNN

## Imports

In [18]:
import pandas as pd
from classes.layers.LSTM import LSTM

## Data and Preprocessing

In [20]:
df = pd.read_csv("/content/gdrive/Shareddrives/IF4074_Tubes 2_13519074/data/ETH-USD/ETH-USD-Test.csv")

# Drop Date Column 
df = df.drop("Date", axis=1)

# Take Last 32 data
df = df.tail(32)

df

,Open,High,Low,Close,Volume
19,1713.962524,1745.779053,1564.031860,1580.787964,23066821734
20,1574.858154,1642.157227,1564.031860,1634.755005,17897150206
21,1635.083008,1648.945679,1466.139404,1471.693481,26946275878
22,1471.928345,1480.564209,1415.581299,1432.447754,16764804299
23,1432.876465,1473.060425,1415.042358,1469.741699,10798098671
24,1469.705444,1469.705444,1331.494751,1335.329102,15762284723
25,1335.270752,1388.272217,1287.420898,1377.541382,18712714223
26,1377.619019,1381.965698,1319.201416,1324.388184,14722317220
27,1324.215576,1384.476929,1229.426758,1252.607788,20643507800
28,1251.567383,1336.157837,1240.968262,1327.680176,18461527259


### Data Shape

In [21]:
data_shape = df.shape
data_shape

(32, 5)

## Model

In [22]:
model = Sequential([
  InputLayer(input_shape=data_shape),
  LSTM(32),
  Dense(5)
])
model.compile(data_shape)

In [23]:
model.summary()

Name        Type        Output Shape      Num of Params
----------  ----------  --------------  ---------------
InputLayer  InputLayer  (None, 32, 5)                 0
LSTM        LSTM        (None, 32)                 8320
Dense       Dense       (None, 5)                   165
Total params:  8485
Trainable params:  8485
Non-trainable params:  0


In [24]:
output = model.predict(df.values)
print(output)

[13.79026525 16.01430911 14.43523891 14.81894576 14.0761636 ]


## Output Analysis

Dari output tersebut, dapat disimpulkan bahwa bahwa prediksi nilai selanjutnya dari kolom Open, High, Low, Close, dan Volume berturut-turut adalah 13.79026525, 16.01430911, 14.43523891, 14.81894576, dan 14.0761636. Tentunya hasil ini memiliki perbedaan yang signifikan dengan data-data sebelumnya. Hal ini dikarenakan model yang digunakan untuk memprediksi output belum di-train dan hanya menggunakan inisialisasi bobot secara acak. Kedepannya, apabila model ini dilakukan proses training, maka diharapkan output yang dihasilkan akan menjadi lebih baik dan relevan dengan data-data sebelumnya, sehingga dapat memberikan prediksi yang relatif lebih akurat.